# Hyperbolic kNN

**Курс:** Рекомендательные системы

**Проект:** Hyperbolic KNN

**Команда:** Сидоров Александр, Сипен, Фролов Александр.

**Код и воспроизводимость:** GitHub: https://github.com/AlexxanderrSid/Hyperbolic-KNN/tree/main

---

## Описание проекта и гипотеза

**Задача.** Исследовать, как меняется качество kNN-рекомендателей, если заменить евклидову/косинусную близость на гиперболическую, и оценить эффект на реальных данных.

**Мотивация.** Гиперболические пространства хорошо описывают иерархические структуры (деревья/графы), а пользовательские предпочтения и совместные покупки часто имеют иерархические паттерны (например, продукты питания → молочные → йогурты). Гипотеза проекта: гиперболическая метрика может лучше сохранять иерархическую структуру и улучшить качество KNN методов.

**Что проверяем.**

1. Качество бейзлайнов (TopPop, UserKNN, ItemKNN, SVD, TIFU-KNN).
2. Качество гиперболических вариантов kNN (Hyperbolic UserKNN, Hyperbolic TIFU-KNN, экспериментально — hyperbolic ItemKNN).
3. Качество vs покрытие каталога.

---

## Данные и сценарий эксперимента

**Датасет:** Ta-Feng (транзакции супермаркета).

**Постановка:** `next-basket recommendation` в warm-start сценарии (у каждого пользователя есть история покупок).

### Формирование корзин

Корзина определяется как множество уникальных товаров, купленных пользователем в один день:

* агрегируем транзакции по `(user, day)` → `basket = set(items)`
* убираем некорректные даты/пропуски
* минимальная длина истории: `MIN_BASKETS_PER_USER ≥ 3` (нужно для train/val/test)

Использовалось **8000 пользователей** и **21149 товаров**.

### Разбиение train/val/test (time-based per user)

Для каждого пользователя (по времени):

* **train:** все корзины, кроме двух последних
* **val:** предпоследняя корзина
* **test:** последняя корзина

---

## Метрики

Оцениваем качество ранжирования рекомендованных товаров относительно истинной корзины пользователя.

### Recall@K

$
Recall@K(u)=\frac{|Rec_u^{K} \cap True_u|}{|True_u|}
$
где $Rec_u^{K}$ — топ-K рекомендаций для пользователя u, $True_u$ — товары в отложенной корзине val/test.

### NDCG@K

$
NDCG@K(u)=\frac{DCG@K(u)}{IDCG@K(u)}, \quad
DCG@K(u)=\sum_{i=1}^{K}\frac{\mathbb{1}[rec_i \in True_u]}{\log_2(i+1)}
$

### Coverage@K

Покрытие каталога долей уникальных рекомендованных товаров на всем множестве пользователей:
$
Coverage@K = \frac{\left|\bigcup\limits_u Rec_u^{K}\right|}{|\mathcal{I}|}
$
где $|\mathcal{I}|$ — число товаров в каталоге.

**Отчётные K:** $K \in {10, 100}$ (дополнительно считались 5/20).

---

## Бейзлайны

Набор бейзлайнов покрывает типовые классы методов из курса и близок к дальнейшей цели проекта (kNN + его гиперболизация).

1. **TopPop** — нижняя граница. Проверяет силу популярностной части.
2. **UserKNN (cosine)** — ключевой baseline для дальнейшего Hyperbolic UserKNN.
3. **TIFU-KNN (simple)** — временно-взвешенный neighbor-based baseline (приближённый к TIFU-подходу).
4. **SVD (TruncatedSVD)** — быстрый baseline, основанный на матричной факторизации.
5. **ItemKNN (cosine)** — снимает неоднозначность KNN baseline (user-based vs item-based) и даёт контрольные значения по coverage.

Сравнение с этими бейзлайнами позволяет:

* Отделить эффект персонализации от популярности
* Понять силу kNN-методов в евклидовой геометрии
* Оценить влияние времени (TIFU) и факторизации (SVD)
* Честно сопоставить гиперболический kNN с обычным kNN

---

## Модели и формализация

### UserKNN (cosine)

Строим user–item матрицу X по train (counts of basket presence).

Косинусная похожесть:
$
sim(u,v)=\frac{\langle x_u, x_v\rangle}{|x_u||x_v|}
$

Score по соседям:
$
score(u,i)=\sum_{v\in N_k(u)} sim(u,v)\cdot X_{v,i}
$

### TIFU-KNN (simple)

Строим два профиля пользователя:

* $IU_u$: бинарная общность (item usage) — встречался ли товар в истории train.
* $PIF_u$: частоты с затуханием по времени по корзинам (Products Interaction Frequency).

Вес корзины во времени:
$
w(t)=\gamma^{d_g(t)}\cdot \delta^{d_w(t)}
$
, где $\gamma$ = group_decay, $\delta$ = within_decay, $d_g, d_w$ — расстояние до последних групп.

Смешивание похожестей:
$
sim(u,v)=\alpha\cdot cos(PIF_u,PIF_v) + (1-\alpha)\cdot cos(IU_u,IU_v)
$
и далее агрегация по соседям аналогично UserKNN.

### SVD (TruncatedSVD на implicit counts)

Факторизация матрицы взаимодействий:
$
X \approx U V^\top
$
Рекомендационный score:
$
score(u,i) = U_u \cdot V_i
$

### Гиперболические варианты kNN

Используем модель Пуанкаре (Poincaré ball): точки $z \in \mathbb{B}^d$, $|z|<1$.

Гиперболическое расстояние:
$
d_{\mathbb{B}}(x,y)=\operatorname{arcosh}\left(1+2\frac{|x-y|^2}{(1-|x|^2)(1-|y|^2)}\right)
$

Далее расстояние превращаем в похожесть:
$
sim(x,y)=\exp(-d_{\mathbb{B}}(x,y)) \quad \text{или} \quad sim(x,y)=\frac{1}{1+d_{\mathbb{B}}(x,y)}
$
и применяем kNN-агрегацию как в обычном UserKNN/TIFU-KNN.

---

## Методология экспериментов

* **Сценарий:** warm-start, next-basket; все пользователи имеют историю.
* **Holdout:** per-user time holdout (val/test — последние корзины).
* **Предобработка:** корзины — уникальные товары.
* **Тюнинг:** на подмножестве пользователей по **NDCG@10**, затем переобучение лучших конфигураций и оценка на full val/test.
* **Метрики:** Recall@K, NDCG@K, Coverage@K для (K={10,100}).

---

## Результаты

### Итоговые результаты бейзлайнов (VAL/TEST, K=10 и K=100)

| Method   | Model                        | NDCG@10 (val) | Recall@10 (val) | Coverage@10 (val) | NDCG@100 (val) | Recall@100 (val) | Coverage@100 (val) | NDCG@10 (test) | Recall@10 (test) | Coverage@10 (test) | NDCG@100 (test) | Recall@100 (test) | Coverage@100 (test) |
| -------- | ---------------------------- | ------------: | --------------: | ----------------: | -------------: | ---------------: | -----------------: | -------------: | ---------------: | -----------------: | --------------: | ----------------: | ------------------: |
| TopPop   | TopPop                       |      0.052924 |        0.051219 |          0.000473 |       0.086659 |         0.164653 |           0.004728 |       0.075881 |         0.067042 |           0.000473 |        0.109748 |          0.181799 |            0.004728 |
| SVD      | SVD(dim=128)                 |      0.068860 |        0.065705 |          0.041610 |       0.093317 |         0.157011 |           0.178543 |       0.075659 |         0.072342 |           0.041610 |        0.098601 |          0.156790 |            0.178543 |
| UserKNN  | UserKNN(k=100)               |  **0.088300** |        0.094446 |          0.141141 |   **0.116596** |         0.196625 |           0.460731 |   **0.099883** |         0.106570 |           0.141141 |    **0.126745** |          0.203505 |            0.460731 |
| TIFU-KNN | TIFU(simple,g=5,a=0.9,k=100) |      0.088114 |    **0.097130** |      **0.149558** |       0.116149 |     **0.198458** |       **0.477138** |       0.097342 |     **0.108195** |       **0.149558** |        0.123973 |      **0.204306** |        **0.477138** |
| ItemKNN  | ItemKNN(topk=200)            |      0.038328 |        0.045689 |          0.810251 |       0.068244 |         0.144216 |           0.912856 |       0.042073 |         0.052969 |           0.810251 |        0.071670 |          0.149403 |            0.912856 |

**Ключевые наблюдения:**

* **UserKNN и TIFU** — лучшие по NDCG/Recall на val и test. Различия небольшие (UserKNN лучше по NDCG, TIFU — по Recall и coverage).
* **TopPop** значительно хуже персонализированных методов и имеет почти нулевой coverage (это по факту фиксированный набор).
* **SVD** улучшает coverage по сравнению с TopPop, но уступает neighbor-based методам по качеству.
* **ItemKNN** даёт крайне высокий coverage (почти весь каталог при K=100), но заметно проигрывает по NDCG/Recall.

### Гиперболические варианты

По реализованным гиперболическим вариантам (K=5/10/20):

* **Hyperbolic UserKNN**

  * VAL: NDCG@10 = 0.059842, Recall@10 = 0.062983
  * TEST: NDCG@10 = 0.085166, Recall@10 = 0.085379

* **Hyperbolic TIFU-KNN**

  * VAL: NDCG@10 = 0.063113, Recall@10 = 0.068223
  * TEST: NDCG@10 = 0.088212, Recall@10 = 0.093088

В текущем виде гиперболические варианты **уступают** сильным евклидовым бейзлайнам (UserKNN/TIFU по cosine). Это важный результат: простая замена метрики без обучения гиперболических эмбеддингов не гарантирует улучшений.

---

## Обсуждение и интерпретация эффектов

1. **Почему kNN работает лучше TopPop/SVD.** Для grocery next-basket характерны повторные покупки, и neighbor-based методы хорошо ловят локальные шаблоны совместных наборов товаров.

2. **TIFU vs UserKNN.**
   TIFU чуть выигрывает по Recall и Coverage (особенно на K=100), что соответствует интуиции: временное затухание усиливает актуальные товары и расширяет разнообразие. При этом UserKNN остаётся сильнее по NDCG@10, то есть немного лучше ранжирует верх списка.

3. **ItemKNN: coverage ≫ качество.**
   ItemKNN даёт очень высокий coverage (до ~0.91 при K=100), но уступает по Recall/NDCG. Это отражает trade-off: модель сильно диверсифицирует выдачу, но хуже попадает в релевантные товары следующей корзины.

4. **Гиперболический kNN: наблюдаемый эффект и выводы.**
   В наших экспериментах гиперболические варианты **уступают** сильным евклидовым бейзлайнам (UserKNN/TIFU на cosine).
   Например, на **VAL**:
   - UserKNN (best) NDCG@10 = **0.088300**, Recall@10 = **0.094446**,
   - Hyperbolic UserKNN NDCG@10 = **0.059842**, Recall@10 = **0.062983**.
   Аналогично для TIFU:
   - TIFU (best) NDCG@10 = **0.088114**, Recall@10 = **0.097130**,
   - Hyperbolic TIFU-KNN NDCG@10 = **0.063113**, Recall@10 = **0.068223**.

   На test гиперболические варианты улучшаются относительно val, но всё равно остаются ниже лидеров:
   - UserKNN NDCG@10 = **0.099883** vs Hyperbolic UserKNN **0.085166**,
   - TIFU NDCG@10 = **0.097342** vs Hyperbolic TIFU-KNN **0.088212**.

---

## Дальнейшие улучшения

1. Подбор/обучение параметров: кривизна, размерность, функция преобразования расстояния в similarity.
2. Ускорение kNN (ANN/FAISS, предрасчёт соседей) — особенно важно для гиперболического расстояния.
3. Валидация на другом датасете (MovieLens) для сравнимости с литературой. Хотя для TIFU и next-basket Ta-Feng выглядит более естественным.

---

## Распределение задач

Сидоров Александр: реализация HyperbolicUserKNN (гиперболическая метрика/соседи, прогон val/test, сравнение с бейзлайнами), оформление результатов по гиперболическому UserKNN.

Сипен: реализация HyperbolicTIFU-KNN (гиперболическая версия TIFU, прогон val/test, сравнение с бейзлайнами), сбор и оформление результатов гиперболических вариантов для отчёта, презентации.

Фролов Александр: пайплайн данных, разбиение train/val/test, бейзлайны (TopPop/UserKNN/SVD/TIFU/ItemKNN), метрики Recall/NDCG/Coverage, таблицы результатов.

Общие задачи: отчёт, презентация, прогон экспериментов

---

## Ссылки на релевантные статьи

* Обзор по гиперболическим представлениям и ML: [https://arxiv.org/pdf/2101.04562](https://arxiv.org/pdf/2101.04562)
* Poincaré embeddings: [https://arxiv.org/pdf/1805.09112](https://arxiv.org/pdf/1805.09112)
* Hyperbolic подходы/графы: [https://proceedings.neurips.cc/paper/2019/file/103303dd56a731e377d01f6a37badae3-Paper.pdf](https://proceedings.neurips.cc/paper/2019/file/103303dd56a731e377d01f6a37badae3-Paper.pdf)
* Hyperbolic representation learning: [https://proceedings.neurips.cc/paper_files/paper/2017/file/59dfa2df42d9e3d41f5b02bfc32229dd-Paper.pdf](https://proceedings.neurips.cc/paper_files/paper/2017/file/59dfa2df42d9e3d41f5b02bfc32229dd-Paper.pdf)
* Hyperbolic SASRec: [https://publications.hse.ru/pubs/share/direct/960861862.pdf](https://publications.hse.ru/pubs/share/direct/960861862.pdf)
* Доп. вдохновение/идеи: [https://arxiv.org/pdf/2308.13279.pdf](https://arxiv.org/pdf/2308.13279.pdf)
* Статьи про графы знаний/рекомендательные системы:

  * [https://arxiv.org/pdf/2504.02589](https://arxiv.org/pdf/2504.02589)
  * [https://arxiv.org/pdf/2508.11978](https://arxiv.org/pdf/2508.11978)

